In [19]:
import torch 
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader,Dataset
import os
import pandas
import numpy
from torchvision  import transforms,datasets
from transformers import AutoImageProcessor, AutoModelForImageClassification
from torch.utils.data import random_split


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

## Transforms

In [33]:
train_transform = transforms.Compose([
   transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

## DataSep

In [34]:
test=datasets.ImageFolder(root="/kaggle/input/brain-tumor-classification-mri/Testing",transform=test_transform)
train=datasets.ImageFolder(root="/kaggle/input/brain-tumor-classification-mri/Training",transform=train_transform)
train_size = int(0.8 * len(train))
val_size = len(train) - train_size

train_dataset, val_dataset = random_split(train, [train_size, val_size])


In [35]:
train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=16,shuffle=True)

In [36]:
test_loader=DataLoader(test,batch_size=16,shuffle=True)

In [37]:
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

num_classes = len(train.classes)  #here is it 4
print(f"Number of classes in dataset: {num_classes}")
print(f"Classes: {train.classes}")
#we adjust classifier head 
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)

model.to(device)


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Number of classes in dataset: 4
Classes: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [38]:
loss_function = torch.nn.CrossEntropyLoss()


In [ ]:
"""class ViTProcessorTransform:   #you can do this or just make sure the transofrms are exactly vits correct ones in the transforms
    def __init__(self, processor):
        self.processor = processor
    
    def __call__(self, image):
        # Convert PIL to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # Use ViT processor
        processed = self.processor(image, return_tensors="pt")
        return processed['pixel_values'].squeeze(0)
transform = transforms.Compose([
    ViTProcessorTransform(processor),
    #+ the augmentations stuff for train transofmrs
])"""



In [43]:
optimizer = Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


In [44]:
train_N = len(train_loader.dataset)
test_N = len(test_loader.dataset)
print(train_N,test_N)

2296 394


In [45]:
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    # training 
    train_loss = 0
    train_correct = 0
    model.train()
    
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        
        output = model(x)
        logits = output.logits
        
        optimizer.zero_grad()
        batch_loss = loss_function(logits, y)
        
        batch_loss.backward()
        optimizer.step()
        
        train_loss += batch_loss.item()
        train_correct += (logits.argmax(dim=1) == y).sum().item()
        
        if batch_idx % 20 == 0:  # we can just write the progres each 20 batch in the epoch
            print(f'Batch {batch_idx}/{len(train_loader)}, Loss: {batch_loss.item():.4f}')
    
    train_accuracy = train_correct / len(train_loader.dataset)
    avg_train_loss = train_loss / len(train_loader)
    print(f'Train - Loss: {avg_train_loss:.4f} Accuracy: {train_accuracy:.4f}')
    
    # validation 
    model.eval()
    val_loss = 0
    val_correct = 0
    
    with torch.no_grad():
        for x, y in val_loader:  
            x, y = x.to(device), y.to(device)
            output = model(x)
            logits = output.logits
            
            val_loss += loss_function(logits, y).item()
            val_correct += (logits.argmax(dim=1) == y).sum().item()
    
    val_accuracy = val_correct / len(val_loader.dataset)
    avg_val_loss = val_loss / len(val_loader)
    print(f'Valid - Loss: {avg_val_loss:.4f} Accuracy: {val_accuracy:.4f}')
    print('-' * 50)

Epoch 1/10
Batch 0/144, Loss: 1.4370
Batch 20/144, Loss: 0.9229
Batch 40/144, Loss: 0.3206
Batch 60/144, Loss: 0.4727
Batch 80/144, Loss: 0.1109
Batch 100/144, Loss: 0.5192
Batch 120/144, Loss: 0.0288
Batch 140/144, Loss: 0.0698
Train - Loss: 0.4051 Accuracy: 0.8432
Valid - Loss: 0.1897 Accuracy: 0.9425
--------------------------------------------------
Epoch 2/10
Batch 0/144, Loss: 0.0055
Batch 20/144, Loss: 0.0560
Batch 40/144, Loss: 0.0037
Batch 60/144, Loss: 0.0140
Batch 80/144, Loss: 0.0242
Batch 100/144, Loss: 0.0034
Batch 120/144, Loss: 0.0885
Batch 140/144, Loss: 0.0373
Train - Loss: 0.0920 Accuracy: 0.9695
Valid - Loss: 0.2224 Accuracy: 0.9408
--------------------------------------------------
Epoch 3/10
Batch 0/144, Loss: 0.1405
Batch 20/144, Loss: 0.0459
Batch 40/144, Loss: 0.0275
Batch 60/144, Loss: 0.0031
Batch 80/144, Loss: 0.0503
Batch 100/144, Loss: 0.1181
Batch 120/144, Loss: 0.0289
Batch 140/144, Loss: 0.0011
Train - Loss: 0.0474 Accuracy: 0.9856
Valid - Loss: 0.1711 

In [46]:
from sklearn.metrics import classification_report
import numpy as np

# Collect all predictions and true labels
all_predictions = []
all_true_labels = []


model.eval()
with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            logits = output.logits
            predictions = logits.argmax(dim=1)

            all_predictions.extend(predictions.cpu().numpy())
            all_true_labels.extend(y.cpu().numpy())
class_names = test_loader.dataset.classes  # Assuming your dataset has class names
print("\nDetailed Classification Report:")
print(classification_report(all_true_labels, all_predictions, 
                          target_names=class_names, digits=4))


Detailed Classification Report:
                  precision    recall  f1-score   support

    glioma_tumor     1.0000    0.2100    0.3471       100
meningioma_tumor     0.7737    0.9217    0.8413       115
        no_tumor     0.5909    0.9905    0.7402       105
 pituitary_tumor     0.9500    0.7703    0.8507        74

        accuracy                         0.7310       394
       macro avg     0.8287    0.7231    0.6948       394
    weighted avg     0.8155    0.7310    0.6907       394



### Bad bad for the glioma so i will try to adjust the class weights